# SEC 10-Q Eval

Evaluating Docugami KG-RAG against OpenAI Assistants Retrieval for this dataset: https://github.com/docugami/KG-RAG-datasets/tree/main/sec-10-q

## Set up Eval

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!rm -rf temp
!git clone https://github.com/docugami/KG-RAG-datasets.git temp

In [3]:
import os
from pathlib import Path
from datetime import datetime

# Important: Create your OpenAI assistant via https://platform.openai.com/playground
#            and put the assistant ID here. Make sure you upload the identical set of
#            files listed below (these files will be uploaded automatically to Docugami)
OPENAI_ASSISTANT_ID = "asst_qY1M0SeFYlmqkEZsMVZX2VAK"

DOCSET_NAME = "SEC 10Q Filings"
EVAL_NAME = DOCSET_NAME + " " + datetime.now().strftime("%Y-%m-%d")
FILES_DIR = Path(os.getcwd()) / "temp/sec-10-q/data/v1/docs"
FILE_NAMES = [
    "2022 Q3 AAPL.pdf",
    "2022 Q3 AMZN.pdf",
    "2022 Q3 INTC.pdf",
    "2022 Q3 MSFT.pdf",
    "2022 Q3 NVDA.pdf",
    "2023 Q1 AAPL.pdf",
    "2023 Q1 AMZN.pdf",
    "2023 Q1 INTC.pdf",
    "2023 Q1 MSFT.pdf",
    "2023 Q1 NVDA.pdf",
    "2023 Q2 AAPL.pdf",
    "2023 Q2 AMZN.pdf",
    "2023 Q2 INTC.pdf",
    "2023 Q2 MSFT.pdf",
    "2023 Q2 NVDA.pdf",
    "2023 Q3 AAPL.pdf",
    "2023 Q3 AMZN.pdf",
    "2023 Q3 INTC.pdf",
    "2023 Q3 MSFT.pdf",
    "2023 Q3 NVDA.pdf",
]

# Using mini set to save cost while developing, use full set for actual runs (~$300 per run in OpenAI costs per run)
GROUND_TRUTH_CSV = Path(os.getcwd()) / "temp/sec-10-q/data/v1/qna_data_mini.csv"

# We will run each experiment multiple times and average,
# since results vary slightly over runs
PER_EXPERIMENT_RUN_COUNT = 5

# Note: Please specify ~6 (or more!) similar files to process together as a document set
# This is currently a requirement for Docugami to automatically detect motifs
# across the document set to generate a semantic XML Knowledge Graph.
assert len(FILE_NAMES) >= 6, "Please provide at least 6 files"

In [10]:
import pandas as pd
from langsmith import Client

# Read
df = pd.read_csv(GROUND_TRUTH_CSV)

# Dataset
client = Client()
dataset_name = EVAL_NAME
existing_datasets = list(client.list_datasets(dataset_name=dataset_name))
if existing_datasets:
    # read existing dataset
    dataset = client.read_dataset(dataset_name=dataset_name)
else:
    dataset = client.create_dataset(dataset_name=dataset_name)
    # Populate dataset
    for _, row in df.iterrows():
        q = row["Question"]
        a = row["Answer"]
        client.create_example(
            inputs={"question": q}, outputs={"answer": a}, dataset_id=dataset.id
        )

## Set up Docugami KG-RAG

#### Upload files to Docugami

In [ ]:
from docugami import Docugami
from docugami.lib.upload import upload_to_named_docset, wait_for_dgml

dg_client = Docugami()
file_paths = [FILES_DIR / file_name for file_name in FILE_NAMES]

# Files will not be re-uploaded if they were previously uploaded (based on name)
dg_docs = upload_to_named_docset(dg_client, file_paths, DOCSET_NAME)

docset_id = ""
docset_name = ""
for doc in dg_docs:
    if not docset_id:
        docset_id = doc.docset.id
    else:
        # all docs must be in the same docset
        assert docset_id == doc.docset.id

    if not docset_name:
        docset_name = dg_client.docsets.retrieve(doc.docset.id).name

In [ ]:
# Wait for files to finish processing (OCR, and zero-shot creation of XML knowledge graph)

# Note: This can take some time on the free docugami tier (up to ~20 mins). Please contact us for faster paid plans.
wait_for_dgml(dg_client, dg_docs)

In [ ]:
# Run indexing
from docugami_kg_rag.indexing import index_docset

assert docset_id
assert docset_name

# Note: This can take some time since it is embedding and creating summaries for all the docs and chunks
index_docset(docset_id=docset_id, name=docset_name)

#### Create Docugami Agent

In [6]:
from docugami_kg_rag.agent import build_agent_runnable
from langchain_core.messages import HumanMessage

def predict_docugami_agent(input: dict, config: dict = None) -> str:
    docugami_agent = build_agent_runnable()
    question = input["question"]
    return docugami_agent.invoke(
        {
            "messages": [HumanMessage(content=question)],
        }
    )

/root/Source/github/langchain-template-docugami-kg-rag/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading default rankgpt3 model for language en
Loading RankGPTRanker model gpt-3.5-turbo


In [8]:
# Test the agent to make sure it is working
predict_docugami_agent({"question": "How much did Microsoft spend for opex in the latest quarter?"})

Parent run c43a0a12-b825-4ed0-bd80-cad6a4a57abc not found for run f4b7d6cd-06f4-43b1-a6b3-5fc8157708d3. Treating as a root run.


'The information provided does not specify the exact amount Microsoft spent on operating expenses (opex) for the latest quarter ended September 30, 2023, but it mentions that the operating expenses increased by $119 million, marking a 2% increase from the previous period. To find the exact amount spent on opex, one would need to look at the specific figures from the previous period and apply the mentioned increase.'

## Set up OpenAI Assistants Retrieval

### Create OpenAI Agent

Please go to https://platform.openai.com/playground and create your agent. 

In [4]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

def predict_openai_agent(input: dict, config: dict = None) -> str:
    openai_agent = OpenAIAssistantRunnable(assistant_id=OPENAI_ASSISTANT_ID, as_agent=True).with_config(config)
    question = input["question"]
    result = openai_agent.invoke({"content": question})

    return result.return_values["output"]

In [5]:
# Test the agent to make sure it is working
predict_openai_agent({"question": "How much did Microsoft spend for opex in the latest quarter?"})

"Microsoft's operating expenses for the latest quarter, which ended on September 30, 2023, increased by $168 million or 1% compared to the previous year.\n\nSOURCE(S): 2023 Q3 MSFT.pdf "

## Run Evals


In [11]:
import uuid
from langsmith.client import Client
from langchain.smith import RunEvalConfig
from langchain.globals import set_llm_cache, get_llm_cache

eval_config = RunEvalConfig(
    evaluators=["qa"],
)


def run_eval(eval_func, eval_run_name):
    """
    Run eval
    """
    client = Client()
    client.run_on_dataset(
        dataset_name=EVAL_NAME,
        llm_or_chain_factory=eval_func,
        evaluation=eval_config,
        verbose=True,
        project_name=eval_run_name,
        concurrency_level=2,  # Reduced to help with rate limits, but will take longer
    )


# Experiments
agent_map = {
    "docugami_kg_rag_zero_shot": predict_docugami_agent,
    "openai_assistant_retrieval": predict_openai_agent,
}

try:
    # Disable global cache setting to get fresh results every time for all experiments
    # since no caching or temperature-0 is supported for the openai assistants API and
    # we want to measure under similar conditions
    cache = get_llm_cache()
    set_llm_cache(None)

    for i in range(PER_EXPERIMENT_RUN_COUNT):
        run_id = str(uuid.uuid4())
        for project_name, agent in agent_map.items():
            run_eval(agent, project_name + "_" + run_id)
finally:
    # Revert cache setting to global default
    set_llm_cache(cache)

View the evaluation results for project 'docugami_kg_rag_zero_shot_1def70be-e363-4459-bf3d-aaf6194e8bd0' at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/a5db8a49-d0eb-4150-83e4-68bf08ad8ebf/compare?selectedSessions=84e6f6e4-6ef8-46e9-8c24-61b9941fa65c

View all tests for Dataset SEC 10Q Filings 2024-05-03 at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/a5db8a49-d0eb-4150-83e4-68bf08ad8ebf
[>                                                 ] 0/9

Parent run 7d23199c-b4a2-4aa5-b10d-4981e912bf04 not found for run 1a663ee0-7a83-427c-b4ca-e10e1d234e6a. Treating as a root run.
Parent run ff0b5085-7a78-47f6-940c-ff3b56f17f43 not found for run 4b20bd8c-eb81-4ef0-a120-8e4528f71df2. Treating as a root run.


[---------->                                       ] 2/9

Parent run 188a3636-5427-465e-ab1d-b853199e483d not found for run c9c7073f-fcb2-477f-8b8e-ddc5717608bc. Treating as a root run.
Parent run 670fec85-bd3f-45d0-8d19-813bc9018eda not found for run f23a87b5-8369-4ff1-9538-953d5bb5f44b. Treating as a root run.


[---------------->                                 ] 3/9

...

Parent run 58e74066-a088-4972-a576-22197f970abf not found for run b33ff39f-9791-49c4-8539-37db00fbeb73. Treating as a root run.


[------------------------------------------------->] 9/9

,feedback.correctness,error,execution_time,run_id
count,9.000000,0,9.000000,9
unique,NaN,0,NaN,9
top,NaN,NaN,NaN,c78f8ea0-04de-4848-ba8e-24c152e5a750
freq,NaN,NaN,NaN,1
mean,0.555556,NaN,1.883969,NaN
std,0.527046,NaN,0.221630,NaN
min,0.000000,NaN,1.631498,NaN
25%,0.000000,NaN,1.769093,NaN
50%,1.000000,NaN,1.820031,NaN
75%,1.000000,NaN,1.974672,NaN


View the evaluation results for project 'openai_assistant_retrieval_e0d8d8e5-94b8-4fa8-829c-92ad7d3f9f4b' at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/a5db8a49-d0eb-4150-83e4-68bf08ad8ebf/compare?selectedSessions=d3fe0fbf-4e05-4e2d-ba06-31b4e72154c6

View all tests for Dataset SEC 10Q Filings 2024-05-03 at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/a5db8a49-d0eb-4150-83e4-68bf08ad8ebf
[--------------------------->                      ] 5/9

KeyboardInterrupt: 